## Mystery 1: The Lingering Inventory

We've found products that are marked as discontinued (SellEndDate is in the past), yet they still appear to have quantity available in inventory. Why is this old stock still listed?

**Objectives:**
* Identify products in `Production.Product` that have a `SellEndDate` in the past.
* Join this with `Production.ProductInventory` to check their quantities.
* Filter for products where the inventory quantity is greater than 0.

**Functional Specification:**
* Select relevant columns (`ProductID`, `Name`, `SellEndDate`) from `Production.Product` (aliased as p).
* Select `Quantity` from `Production.ProductInventory` (aliased as inv).
* Use an `INNER JOIN` between `Production.Product` and `Production.ProductInventory` on `p.ProductID = inv.ProductID`.
* Use a `WHERE` clause to filter for products meeting the criteria.
* Add a condition `p.SellEndDate IS NOT NULL`.
* Add a condition `p.SellEndDate < GETDATE()` to find products already discontinued.
* Add a condition `inv.Quantity > 0` to find products with remaining inventory.

In [ ]:
SELECT
    p.ProductID,
    p.Name,
    p.SellEndDate,
    inv.LocationID,
    inv.Quantity
FROM Production.Product AS p
INNER JOIN Production.ProductInventory AS inv
    ON p.ProductID = inv.ProductID
WHERE p.SellEndDate IS NOT NULL
  AND p.SellEndDate < GETDATE()
  AND inv.Quantity > 0;

## Mystery 2: The Inconsistent Costing

We suspect some products that seem like simple variations of each other (e.g., different sizes or colors of the same base model, indicated by similar names) have surprisingly different manufacturing costs.

**Objectives:**
* Identify pairs of products (`ProductID`) that have similar names (e.g., matching the first 15 characters).
* Compare their `StandardCost` to find significant differences (e.g., > 10% variation).

**Functional Specification:**
* Select relevant columns (`ProductID`, `Name`, `StandardCost`) from `Production.Product`.
* Use a self-join on `Production.Product` (aliased as p1 and p2) to compare products.
* Join condition (`ON`): `LEFT(p1.Name, 15) = LEFT(p2.Name, 15)` (Matching start of name) AND `p1.ProductID < p2.ProductID` (avoid self/duplicate pairs).
* `WHERE` clause condition: Check for significant cost difference: `ABS(p1.StandardCost - p2.StandardCost) > (p1.StandardCost * 0.10)`.
* Ensure costs are positive for valid comparison.

In [ ]:
SELECT
    p1.ProductID AS ProductID1,
    p1.Name AS Name1,
    p1.StandardCost AS Cost1,
    p2.ProductID AS ProductID2,
    p2.Name AS Name2,
    p2.StandardCost AS Cost2
FROM Production.Product AS p1
INNER JOIN Production.Product AS p2 ON LEFT(p1.Name, 15) = LEFT(p2.Name, 15)
    AND p1.ProductID < p2.ProductID
WHERE p1.StandardCost > 0 AND p2.StandardCost > 0
  AND ABS(p1.StandardCost - p2.StandardCost) > (p1.StandardCost * 0.10);

## Mystery 3: The Cost Creep Detection

Some product costs seem volatile. Identify any currently sold product whose standard manufacturing cost has jumped by more than 25% between consecutive cost history records.

**Objectives:**
* Query the `Production.ProductCostHistory`.
* For each *currently sold* product, compare its `StandardCost` to the previous cost record's `StandardCost`.
* Identify instances where the cost increased by more than 25%.

**Functional Specification:**
* Use a Common Table Expression (CTE) `CostHistoryWithLag` to calculate the previous cost for each product's cost history entry.
    * Inside the CTE, select `ProductID`, `StartDate`, `StandardCost` from `Production.ProductCostHistory`.
    * Use the `LAG()` window function partitioned by `ProductID` and ordered by `StartDate` to get the `PreviousCost`.
* Select relevant columns from the CTE.
* Join CTE with `Production.Product` (p) on `ProductID` to filter for currently sold items.
* Use a `WHERE` clause to filter for rows where the `StandardCost` is significantly higher than the `PreviousCost`.
    * Condition: `StandardCost > PreviousCost * 1.25`
    * Ensure `PreviousCost` is not NULL and not zero.
    * Ensure product is currently sold (`p.SellEndDate IS NULL OR p.SellEndDate > GETDATE()`).

In [ ]:
WITH CostHistoryWithLag AS (
    SELECT
        ProductID,
        StartDate,
        StandardCost,
        LAG(StandardCost, 1, NULL) OVER (PARTITION BY ProductID ORDER BY StartDate) AS PreviousCost
    FROM Production.ProductCostHistory
)
SELECT
    ch.ProductID,
    p.Name,
    ch.StartDate AS ChangeDate,
    ch.PreviousCost,
    ch.StandardCost
FROM CostHistoryWithLag ch
INNER JOIN Production.Product p ON ch.ProductID = p.ProductID
WHERE ch.PreviousCost IS NOT NULL
  AND ch.PreviousCost > 0
  AND ch.StandardCost > (ch.PreviousCost * 1.25)
  AND (p.SellEndDate IS NULL OR p.SellEndDate > GETDATE());

## Mystery 4: The Mystery of the Unpopular Products

We have products listed for sale, but some haven't appeared on a single sales order. Are these truly unpopular, or is there a data issue?

**Objectives:**
* Identify all products currently marked as available for sale.
* Identify all products that have ever been sold (appeared in `Sales.SalesOrderDetail`).
* Find the products from the first list that are NOT in the second list.

**Functional Specification:**
* Query 1: Use a CTE `CurrentlySoldProducts` to get IDs of products available for sale.
    * Select `ProductID` from `Production.Product`.
    * Filter `WHERE FinishedGoodsFlag = 1` AND (`SellEndDate IS NULL` OR `SellEndDate > GETDATE()`).
* Query 2: Use a CTE `ProductsEverSold` to get IDs of products that have appeared in sales details.
    * Select DISTINCT `ProductID` from `Sales.SalesOrderDetail`.
* Query 3: Find the difference using `EXCEPT` or `NOT EXISTS`.
    * Select `csp.ProductID`, `p.Name` from `CurrentlySoldProducts` (csp).
    * Join with `Production.Product` (p) on `ProductID` to get the name.
    * Use `EXCEPT` to remove products found in `ProductsEverSold`.
    * -- OR --
    * Use `WHERE NOT EXISTS` to check if the `csp.ProductID` is in `ProductsEverSold` (subquery).

In [ ]:
WITH CurrentlySoldProducts AS (
    SELECT ProductID
    FROM Production.Product
    WHERE FinishedGoodsFlag = 1
      AND (SellEndDate IS NULL OR SellEndDate > GETDATE())
),
ProductsEverSold AS (
    SELECT DISTINCT ProductID
    FROM Sales.SalesOrderDetail
)
SELECT
    csp.ProductID,
    p.Name
FROM CurrentlySoldProducts csp
INNER JOIN Production.Product p ON csp.ProductID = p.ProductID
EXCEPT
SELECT
    pes.ProductID,
    p.Name
FROM ProductsEverSold pes
INNER JOIN Production.Product p ON pes.ProductID = p.ProductID
ORDER BY ProductID;

## Mystery 5: The Vendor Over-Reliance

Are we overly reliant on certain vendors? Identify vendors who supply a significantly higher number of distinct products than the average vendor.

**Objectives:**
* Calculate the average number of distinct products supplied per vendor based on purchase history.
* Identify vendors supplying more than, say, twice the average number of distinct products.

**Functional Specification:**
* Query 1: Use CTE `VendorProductCounts` to count distinct products per vendor from purchase history.
    * Select `poh.VendorID`, `COUNT(DISTINCT pod.ProductID)` as `DistinctProductCount`.
    * From `Purchasing.PurchaseOrderDetail` (pod).
    * Join with `Purchasing.PurchaseOrderHeader` (poh) on `PurchaseOrderID`.
    * Group by `poh.VendorID`.
* Query 2: Use CTE `AverageVendorProductCount` to calculate the overall average.
    * Select `AVG(CAST(DistinctProductCount AS DECIMAL(10,2)))` as `AvgCount` from `VendorProductCounts`.
* Query 3: Final Selection.
    * Select `vpc.VendorID`, `v.Name` as `VendorName`, `vpc.DistinctProductCount`.
    * From `VendorProductCounts` (vpc).
    * Join with `Purchasing.Vendor` (v) on `vpc.VendorID = v.BusinessEntityID`.
    * Cross Join with `AverageVendorProductCount` to make the average available.
    * Filter `WHERE vpc.DistinctProductCount > (avg_calc.AvgCount * @AvgThresholdMultiplier)`.
    * Order by `DistinctProductCount` descending.

In [ ]:
DECLARE @AvgThresholdMultiplier_M5 DECIMAL(5,2) = 2.0;

WITH VendorProductCounts AS (
    SELECT
        poh.VendorID,
        COUNT(DISTINCT pod.ProductID) AS DistinctProductCount
    FROM Purchasing.PurchaseOrderDetail AS pod
    INNER JOIN Purchasing.PurchaseOrderHeader AS poh ON pod.PurchaseOrderID = poh.PurchaseOrderID
    GROUP BY poh.VendorID
),
AverageVendorProductCount AS (
    SELECT AVG(CAST(DistinctProductCount AS DECIMAL(10,2))) AS AvgCount
    FROM VendorProductCounts
    WHERE DistinctProductCount > 0
)
SELECT
    vpc.VendorID,
    v.Name AS VendorName,
    vpc.DistinctProductCount
FROM VendorProductCounts AS vpc
INNER JOIN Purchasing.Vendor AS v ON vpc.VendorID = v.BusinessEntityID
CROSS JOIN AverageVendorProductCount AS avg_calc
WHERE vpc.DistinctProductCount > (avg_calc.AvgCount * @AvgThresholdMultiplier_M5)
ORDER BY vpc.DistinctProductCount DESC;

## Mystery 6: The Mystery of the Long Lead Times

Some vendors consistently take a long time to ship orders after they are placed. Identify vendors with average lead times (ShipDate - OrderDate) significantly longer than the norm.

**Objectives:**
* For each vendor, calculate the average number of days between `OrderDate` and `ShipDate` for shipped orders.
* Calculate the overall average lead time across all vendors.
* Identify vendors whose average lead time is significantly above the overall average (e.g., > 1.1 times) and have shipped a minimum number of orders (e.g., > 2).

**Functional Specification:**
* Query 1: Use CTE `VendorLeadTimes` to calculate average lead time per vendor.
    * Select `VendorID`.
    * Calculate `AVG(DATEDIFF(day, OrderDate, ShipDate))` as `AvgLeadTimeDays`.
    * Count `COUNT(*)` as `ShippedOrderCount`.
    * From `Purchasing.PurchaseOrderHeader`.
    * Filter `WHERE ShipDate IS NOT NULL`.
    * Group by `VendorID`.
* Query 2: Use CTE `OverallAverageLeadTime` to calculate the overall average.
    * Select `AVG(AvgLeadTimeDays)` as `OverallAvg` from `VendorLeadTimes`.
* Query 3: Final Selection.
    * Select `vlt.VendorID`, `v.Name` as `VendorName`, `vlt.ShippedOrderCount`, `CAST(vlt.AvgLeadTimeDays AS DECIMAL(10,2))` as `AverageLeadTime`.
    * From `VendorLeadTimes` (vlt).
    * Join with `Purchasing.Vendor` (v) on `vlt.VendorID = v.BusinessEntityID`.
    * Cross Join with `OverallAverageLeadTime` (oalt).
    * Filter `WHERE vlt.AvgLeadTimeDays > (oalt.OverallAvg * @LeadTimeMultiplier)` AND `vlt.ShippedOrderCount >= @MinShippedOrders`.
    * Order by `AverageLeadTime` descending.

In [ ]:
DECLARE @MinShippedOrders_M6 INT = 2;
DECLARE @LeadTimeMultiplier_M6 DECIMAL(5,1) = 1.1;

WITH VendorLeadTimes AS (
    SELECT
        VendorID,
        AVG(CAST(DATEDIFF(day, OrderDate, ShipDate) AS DECIMAL(10,2))) AS AvgLeadTimeDays,
        COUNT(*) AS ShippedOrderCount
    FROM Purchasing.PurchaseOrderHeader
    WHERE ShipDate IS NOT NULL
      AND OrderDate IS NOT NULL
      AND ShipDate >= OrderDate
    GROUP BY VendorID
),
OverallAverageLeadTime AS (
    SELECT AVG(AvgLeadTimeDays) AS OverallAvg
    FROM VendorLeadTimes
    WHERE ShippedOrderCount >= @MinShippedOrders_M6
)
SELECT
    vlt.VendorID,
    v.Name AS VendorName,
    vlt.ShippedOrderCount,
    CAST(vlt.AvgLeadTimeDays AS DECIMAL(10,2)) AS AverageLeadTime
FROM VendorLeadTimes vlt
INNER JOIN Purchasing.Vendor v ON vlt.VendorID = v.BusinessEntityID
CROSS JOIN OverallAverageLeadTime oalt
WHERE vlt.ShippedOrderCount >= @MinShippedOrders_M6
  AND vlt.AvgLeadTimeDays > (oalt.OverallAvg * @LeadTimeMultiplier_M6)
ORDER BY AverageLeadTime DESC;

## Mystery 7: The Mystery of the Solo Customer Territories

Analysis suggests some sales territories have only one assigned customer. Is this accurate, indicating low penetration, or perhaps a data entry error assigning customers?

**Objectives:**
* Count the number of customers assigned to each sales territory.
* Identify territories that have exactly one customer.
* Display the territory name and the ID of the single customer in that territory.

**Functional Specification:**
* Query 1: Use a CTE `TerritoryCustomerCount` to find territories with only one customer.
    * Select `TerritoryID`, `COUNT(*)` as `CustomerCount`, `MIN(CustomerID)` as `SingleCustomerID`.
    * From `Sales.Customer`.
    * Filter `WHERE TerritoryID IS NOT NULL`.
    * Group by `TerritoryID`.
    * Filter `HAVING COUNT(*) = 1`.
* Query 2: Final selection joining to get territory name.
    * Select `tcc.TerritoryID`, `st.Name` as `TerritoryName`, `tcc.SingleCustomerID`.
    * From `TerritoryCustomerCount` (tcc).
    * Join with `Sales.SalesTerritory` (st) on `tcc.TerritoryID = st.TerritoryID`.
    * Order by `TerritoryName`.

In [ ]:
WITH TerritoryCustomerCount AS (
    SELECT
        TerritoryID,
        COUNT(*) AS CustomerCount,
        MIN(CustomerID) AS SingleCustomerID
    FROM Sales.Customer
    WHERE TerritoryID IS NOT NULL
    GROUP BY TerritoryID
    HAVING COUNT(*) = 1
)
SELECT
    tcc.TerritoryID,
    st.Name AS TerritoryName,
    tcc.SingleCustomerID
FROM TerritoryCustomerCount AS tcc
INNER JOIN Sales.SalesTerritory AS st ON tcc.TerritoryID = st.TerritoryID
ORDER BY st.Name;